# A Hubble Source Catalog (HSC) Use Case

- [Example #2: Using CASJOBS to Query the HSC][1]
  - (Globular Clusters in M87 and a Color Magnitude Diagram for the SMC)
  
![m87_2plot][2]


  [1]: https://archive.stsci.edu/hst/hsc/help/use_case_2_v3.html
  [2]: screenshots/m87_2plot.png

CasJobs is an online tool used to query large databases. Originally created for the [SDSS][1] catalog, it has now been adapted for selected [MAST][2] archival datasets - namely GALEX, Kepler and the HSC. While the [MAST Discovery Portal][3] is the primary tool for using the HSC, CasJobs provides a powerful tool for looking at larger datasets and making more detailed queries. The [HSC Homepage][4] is also available for some specific queries.

<span style="color:red;">GOAL</span>: This tutorial provides a first look at how to use CasJobs to query the Hubble Source Catalog. For a more detailed tutorial on the syntax behind the queries the [SDSS SQL Tutorial][5] is recommended.

<span style="color:red;">SCIENCE CASE</span>: The science cases are studies of: 1. slightly extended objects (i.e., globular clusters in M87) and 2. large datasets (i.e., a Color Magntiude Diagram for the SMC).


  [1]: http://www.sdss.org/
  [2]: http://archive.stsci.edu/
  [3]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [4]: https://archive.stsci.edu/hst/hsc/
  [5]: http://cas.sdss.org/dr7/en/help/howto/search/

In [ ]:
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

from PIL import Image
from io import BytesIO

from astroquery.mast import Observations
import astropy
from astropy.coordinates import SkyCoord
from astropy.table import Table, join
from astropy.io import ascii

# check that version of mastcasjobs is new enough
# we are using some features not in version 0.0.1
from pkg_resources import get_distribution
from packaging.version import Version as V

assert V(get_distribution("mastcasjobs").version) >= V('0.0.2'), """
A newer version of mastcasjobs is required.
Update mastcasjobs to current version using this command:
pip install --upgrade git+git://github.com/rlwastro/mastcasjobs@master
"""

import mastcasjobs

# set width for pprint
astropy.conf.max_width = 150

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import image search utilites from local py file
from image_utils import *

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

In [ ]:
HSCContext = "HSCv3"

Set up Casjobs environment.

In [ ]:
import getpass
if not os.environ.get('CASJOBS_USERID'):
    os.environ['CASJOBS_USERID'] = input('Enter Casjobs UserID:')
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = getpass.getpass('Enter Casjobs password:')

## <span style="color:red;">Step 1</span> - HSC Casjobs

Enter the [HSC Casjobs webpage][1] and create an account using the button in the top menu bar (<span style="color:blue;">blue</span>). Go ahead and login (<span style="color:green;">green</span>).

![welcome][2]


  [1]: https://mastweb.stsci.edu/hcasjobs/
  [2]: screenshots/hsc_welcome.png

## <span style="color:red;">Step 2</span> - Inspect the HSC databases and functions.

Click *MyDB* in the top menu bar (<span style="color:blue;">blue</span>). This displays all the tables and functions you create in "MyDB" (My Database). Now change the "context" (the database used to run your query) from MyDB to HSCv3 (i.e., Hubble Source Catalog Version 3) (<span style="color:green;">green</span>), using the dropdown menu on the top left. This shows the available views, tables, functions and procedures related to the HSC database.

For example, click *Tables* (<span style="color:green;">green</span>), and then select *SumMagAper2Cat* (i.e., the HSC Summary Form using magaper2 magnitudes) (<span style="color:orange;">orange</span>). Scroll across to see the fields located in this table, such as Match ID (<span style="color:yellow;">yellow</span>), the instrument and filter used (i.e., W2_F122M is the magaper2 magnitude using the Wide Field Planetary Camera 2 image and the F122M filter) (<span style="color:yellow;">yellow</span>). A description of the table and some of the columns are provided. You might want to examine the Function *SearchSumCatalog* also, since you will be using it in a minute.

![hsc_tables][1]


 [1]: screenshots/hsc_tables.png

### sample code to update

In [ ]:
target = 'M87'
coord_m87 = SkyCoord.from_name(target)

ra_m87 = coord_m87.ra.degree
dec_m87 = coord_m87.dec.degree
radius = 500 # arcsec

print(f'ra: {ra_m87}\ndec: {dec_m87}')

In [ ]:
HSCContext = "HSCv3"
jobs = mastcasjobs.MastCasJobs(context="MyDB")

## Get histogram of number of images for each source in the region

In [ ]:
query = f"""
SELECT numimages, count(*) as n
FROM SearchSumCatalog({ra_m87}, {dec_m87}, {radius}, 1)
WHERE W2_F606W is not null and W2_F814W is not null
GROUP BY numimages
"""

htab = jobs.quick(query, task_name="HCV demo", context=HSCContext)
htab

## Plot the histogram

In [ ]:
g = sns.relplot(kind='line', data=htab.to_pandas(), x='numimages', y='n', drawstyle='steps-mid', aspect=1.5)

## Get list of sources

Set `numimages` threshold high enough to weed out artifacts.

Note we are retrieving the table directly from the `quick` Casjobs query.  A bigger table could be written to a `MyDB` table in the user area and then retrieved using `jobs.fast_table(tablename)`.

In [ ]:
numimages = 50

query = f"""
SELECT
      MatchRA, MatchDEC, MatchID, CI, W2_F606W, W2_F814W, V_I=W2_F606W - W2_F814W
FROM 
      SearchSumCatalog({ra_m87}, {dec_m87}, {radius}, 1)
WHERE CI > 1.05 and CI < 1.5
      and (W2_F606W - W2_F814W) > 0.0 and (W2_F606W - W2_F814W) < 1.0
      and numimages > {numimages}
ORDER BY matchID 
"""

t0 = time.time()
tab = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec ({len(tab)} rows)")

# clean up the output format
tab['W2_F606W'].format = "{:.3f}"
tab['W2_F814W'].format = "{:.3f}"
tab['V_I'].format = "{:.3f}"
tab['CI'].format = "{:.3f}"
tab['MatchRA'].format = "{:.6f}"
tab['MatchDEC'].format = "{:.6f}"

#tab = tab.to_pandas()
#tab.head()
tab[:10]

## <span style="color:red;">Step 3</span> - Query the database

The query below is designed to find globular clusters in the galaxy M87.

```sql
SELECT
      MatchRA, MatchDEC, MatchID, CI, W2_F606W, W2_F814W, V_I=W2_F606W - W2_F814W
FROM 
      SearchSumCatalog(187.706, 12.391, 500.0, 1)
WHERE CI > 1.05 and CI < 1.5
      and (W2_F606W - W2_F814W) > 0.0 and (W2_F606W - W2_F814W) < 1.0
      and numimages > 50
ORDER BY matchID 
```

The SELECT command defines the columns to include in the output table (such as the RA, DEC, ID, Concentration Index, and magnitudes).

The FROM command defines the query itself, and is composed of the following parts:

a) The HSC function SearchSumCatalog, which is used to search the HSC around the position of M87 (187.706, 12.391) with a specified search radius (500 arcsec), and retrieve the magaper2 (magtype = 1, i.e., aperture magnitudes within a 3 pixel = 0.30 arcsec radius).

b) The WHERE clause defines the search parameters
     i) concentration index (CI = difference between magnitudes for apertures with a radius of 0.10 and 0.30 arcsec) with values between 1.05 and 1.5 (i.e., slightly extended)
     ii) V - I (W2_F606W - W2_F814W) color betwen 0.0 and 1.0
     iii) number of images in a match (>50).

The ORDER BY defines the order of the results (i.e. by matchID).

Now click *Query*. Change the context from MyDB to HSCv3 (if necessary) (<span style="color:blue;">blue</span>). <span style="color:blue;">Note that forgetting to change the context is the most common error causing CasJobs queries to fail</span>. Copy and paste the text from above into the blank query text box (<span style="color:green;">green</span>). Click *Quick* (<span style="color:orange;">orange</span>) to submit the job.

Since this is a *Quick* submission, the table is not saved to MyDB, but is instead listed below. If you want to save the table, add "INTO M87_V3" at the end of the query (see Step 7).

The table can also be downloaded to a local file using the *Save As* button (<span style="color:yellow;">yellow</span>) at the bottom of the page.

Another option is the Submit button, which is generally used for longer queries. The table is saved using the name in the box under "Table (optional)", which defaults to MyTable; change this to M87_V3 for our example. Try this. To see the table click the *MyDB* tab, select the *MyDB* context (if necessary), click on M87_V3, and click on *Sample* to see the file.

![m87_casjob_query][1]


  [1]: screenshots/m87_casjob_query.png

## <span style="color:red;">Step 4</span> - Examine an image

It is a good idea to look at the HSC overlayed on an image, to check for artifacts and uniformity. Copy the *MatchRA* and *MatchDec* values of the 7th source from your table. Open the [HLA][1] and paste those values into the search box (<span style="color:blue;">blue</span>). Click *Search*.

Note: The HLA displays your search coordinates in both decimal degrees and sexidecimal format (<span style="color:blue;">blue</span>). This is useful if you want to find a specific object in the Interactive Display.

In the *Detector* field enter \*acs/wfc\* (<span style="color:green;">green</span>) to select just the ACS Wide Field Camera images. Click on the Display (<span style="color:orange;">orange</span>) button for the F814W/F606W (color) image (or click on the Images button if you would like to look at previews of all the images).

![m87_hla_inventory][2]


  [1]: http://hla.stsci.edu/hlaview.html
  [2]: screenshots/m87_hla_inventory.png

## <span style="color:red;">Step 5</span> - Determine the value of NumImages to use

*Click HSC controls, Require NumImages > 10*, and then the HSC (<span style="color:blue;">blue</span>) button to overlay the HSC on this particular image. We find that NumImages > 10 leaves a number of artifacts and apparently blank circles. If you click on a few of the real objects (<span style="color:green;">green</span>) you find they all have more than NumImages = 50. Some have over 200!

Go back and change to *Require NumImages > 60* (<span style="color:orange;">orange</span>) to remove essentially all of the artifacts (although you are still missing some objects near the center of the galaxy).

*DETERMINING THE BEST VALUE OF NUMIMAGES IS OFTEN ONE OF THE MOST IMPORTANT DECISIONS YOU WILL HAVE WHEN USING THE HSC!*

It is also interesting to note that there are 4 Gaia sources in the image, and they are all well aligned with HSC sources. Clicking on an object brings up the information for both the HSC and Gaia sources.

![m87_2plot][1]


  [1]: screenshots/m87_2plot.png

## <font color="red"> Rick notes</font>

* Most of the information here applied to Pan-STARRS (PS1) images and is irrelevant for HST images.

* The search radius is much bigger than the image requires.  Probably it would be better to compute the search radius from the image coordinates.

* I'm not using this `zoom` value -- it gets reset below.

In [ ]:
filters = "gri"
size_deg = 0.2
zoom = 0.33

pixel_size = 0.25 # PS1 pixel size in arcsec
# get size in pixels from image size in degrees
size = int(size_deg*3600/pixel_size)

In [ ]:
# get a list of HAP images from astroquery.mast
c = SkyCoord(ra_m87, dec_m87, unit='deg')
obstab = Observations.query_criteria(coordinates=c, provenance_name='HAP', radius=0.0)
print(f"Found {len(obstab)} HAP observations")
# select observations for an example image
otab = obstab[np.char.startswith(obstab['obs_id'],'hst_10543_01') & (obstab['filters']!='detection')]
print(f"Found {len(otab)} observations for visit hst_10543_01")
# sort from red to blue
otab = otab[np.argsort(0.5*(otab['em_min']+otab['em_max']))[::-1]]
otab['obs_id', 'em_min', 'em_max', 'filters']

In [ ]:
# color image
# imtitle = '/'.join(otab['obs_id'])
# gim = getjpegim(otab['obs_id'])

# grayscale image
imtitle = otab['obs_id'][0]
gim = getjpegim(otab['obs_id'][:1], autoscale=98.0)

print(f"Got {gim.size} jpeg image")

# get Gaia EDR3 catalog (search a circle that just touches the image edges)
gcat = getgaia(ra_m87, dec_m87, radius=size_deg/2)
print(f"Got {len(gcat)} Gaia EDR3 sources")

# extract image WCS
iwcs = getwcs(gim)
gcat['x'], gcat['y'] = iwcs.wcs_world2pix(gcat['ra'],gcat['dec'],0)

In [ ]:

fig, ax = plt.subplots(figsize=(10, 10))

# note we must specify the extent to get the pixel origin in the lower left corner
# The cmap applies only if the JPEG is a grayscale image.
ax.imshow(gim, origin="upper", extent=(0, gim.size[0], 0, gim.size[1]), cmap="afmhot")

ax.set_title(f'{target} {imtitle} (jpg)')

# fix limits using the image rather than the catalog
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot('x', 'y', 'o', markerfacecolor='none', data=gcat)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.show()

## <span style="color:red;">Step 6</span>

You can also use the MAST Discovery Portal to make the same query, since this is a relatively small dataset. Several other HSC Use Cases (e.g., #1: Using the Discovery Portal to Query the HSC - Stellar Photometry in M31; Brown et al. 2009) provide more detail. The Discovery Portal can also be used to make plots and to reach the HLA Interactive Display (like in Step 5).

In this example we center on the coordinates of the object (<span style="color:blue;">blue</span>) in the list from the CasJobs search in Step 3 to find the object with matchID = 409428 (<span style="color:orange;">orange</span>). Also note that M87 has been visited 111 times using 216 different images (<span style="color:green;">green</span>) in this region!

![m87_dp][1]


  [1]: screenshots/m87_dp.png

In [ ]:
target = 'M87'
coord_m87 = SkyCoord.from_name(target)

ra_m87 = coord_m87.ra.degree
dec_m87 = coord_m87.dec.degree
radius = 120 # arcsec

print(f'ra: {ra_m87}\ndec: {dec_m87}')

In [ ]:
HSCContext = "HSCv3"

DBtable = "m87"
jobs = mastcasjobs.MastCasJobs(context="MyDB", request_type='POST')

# drop table if it already exists
jobs.drop_table_if_exists(DBtable)

query = f"""
SELECT
      MatchRA, MatchDEC, MatchID, CI, A_F555W, A_F814W, V_I=(A_F555W - A_F814W), TargetName
INTO
      mydb.{DBtable}
FROM 
     SearchSumCatalog({ra_m87},{dec_m87},{radius},1) 
WHERE numimages >= 60 AND numimages <= 216
ORDER BY matchID
"""

# add the following line at the bottom of the query, to filter the results
# where W2_F555W is not null and W2_F814W is not null

t0 = time.time()
results = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec")
print(results)

# fast retrieval using special MAST Casjobs service
tab = jobs.fast_table(DBtable, verbose=True)

tab = tab.to_pandas()
tab.head()

In [ ]:
tab[tab.MatchID.eq(7978688)]

In [ ]:
g = sns.relplot(data=tab, x='MatchRA', y='MatchDEC', color='purple', height=8, alpha=.5, hue='TargetName')

# extract the axes for the figure
ax = g.axes.flat[0]

sns.scatterplot(data=tab[tab.MatchID.eq(7978688)], x='MatchRA', y='MatchDEC', color='r', marker='X', alpha=0.5, ax=ax, s=100, label='MatchID: 409428')

# remove the legend for g
g._legend.remove()

# move the legend for 408428
sns.move_legend(ax, loc='center left', bbox_to_anchor=(1, 0.5), title='Target Name', frameon=False)

g.fig.suptitle(target, y=1.05)

for ax in g.axes.ravel():
    ax.invert_xaxis()

### Below here can be deleted

In [ ]:
tab

In [ ]:
c

In [ ]:
filters = "gri"
size_deg = 0.2
zoom = 0.33

pixel_size = 0.25 # PS1 pixel size in arcsec
# get size in pixels from image size in degrees
size = int(size_deg*3600/pixel_size)

In [ ]:
# get a list of HAP images from astroquery.mast
c = SkyCoord(ra_m31, dec_m31, unit='deg')
obstab = Observations.query_criteria(coordinates=c, provenance_name='HAP', radius=0.0)
print(f"Found {len(obstab)} HAP observations")
# select observations for an example image
# otab = obstab[np.char.startswith(obstab['obs_id'],'hst_10543_01') & (obstab['filters']!='detection')]
# print(f"Found {len(otab)} observations for visit hst_10543_01")
# sort from red to blue
# otab = otab[np.argsort(0.5*(otab['em_min']+otab['em_max']))[::-1]]
# otab['obs_id', 'em_min', 'em_max', 'filters']

obstab['obs_id', 'em_min', 'em_max', 'filters']

In [ ]:
otab = obstab.to_pandas()

In [ ]:
otab.filter('Id')

In [ ]:
# color image
# imtitle = '/'.join(otab['obs_id'])
# gim = getjpegim(otab['obs_id'])

# grayscale image
imtitle = otab['obs_id'][0]
gim = getjpegim(otab['obs_id'][:1], autoscale=98.0)

print(f"Got {gim.size} jpeg image")

# get Gaia EDR3 catalog (search a circle that just touches the image edges)
gcat = getgaia(ra_m31, dec_m31, radius=size_deg/2)
print(f"Got {len(gcat)} Gaia EDR3 sources")

# extract image WCS
iwcs = getwcs(gim)
gcat['x'], gcat['y'] = iwcs.wcs_world2pix(gcat['ra'], gcat['dec'], 0)

In [ ]:
gcat = gcat.to_pandas()

In [ ]:
gcat.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# note we must specify the extent to get the pixel origin in the lower left corner
# The cmap applies only if the JPEG is a grayscale image.
ax.imshow(gim, origin="upper", extent=(0, gim.size[0], 0, gim.size[1]), cmap="afmhot")

ax.set_title(f'{target} {imtitle} (jpg)')

# fix limits using the image rather than the catalog
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot('x', 'y', 'o', markerfacecolor='none', data=gcat)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.show()